In [1]:
import time
import math
import numpy as np
import pandas as pd
from scipy.stats import circmean
import matplotlib.pyplot as plt
import sqlite3
import csv
import json

con = sqlite3.connect('../summer')
cur = con.cursor()

In [3]:
'''
TAs = ['\'Mark\'', '\'vhil\'', '\'record\'', '\'Eugy\'']

recordings_train = ['\'1\'', '\'2\'', '\'3\'', '\'4\'', '\'5\'', '\'6\'']
recordings_test = ['\'7\'', '\'8\'']

print('start time: ', time.strftime("%H:%M:%S", time.localtime()))
all_weeks_train = pd.read_sql_query("""SELECT PlayerName, RecordingName,
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName NOT IN ({})
    AND Week IN ({})
    """.format(",".join(TAs), ','.join(recordings_train)), con)

print('end train time: ', time.strftime("%H:%M:%S", time.localtime()))

all_weeks_test = pd.read_sql_query("""SELECT PlayerName, RecordingName,
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName NOT IN ({})
    AND Week IN ({})
    """.format(",".join(TAs), ','.join(recordings_test)), con)

print('end test time: ', time.strftime("%H:%M:%S", time.localtime()))

print(all_weeks_train.shape)
print(all_weeks_test.shape)

all_weeks_train.to_csv('random_forest/train_df_all.csv', index=False)
all_weeks_test.to_csv('random_forest/test_df_all.csv', index=False)
'''

start time:  22:47:42
end train time:  23:57:16
end test time:  00:15:39
(41600721, 23)
(14009822, 23)


In [2]:
all_weeks_train = pd.read_csv('random_forest/train_df_all.csv')
all_weeks_test = pd.read_csv('random_forest/test_df_all.csv')

print(all_weeks_train.shape)
print(all_weeks_test.shape)

/var/folders/wc/4gk2k67x5pq3tgttzqdnbkkw0000gn/T/ipykernel_78238/420462767.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  all_weeks_train = pd.read_csv('random_forest/train_df_all.csv')


(41600721, 23)
(14009822, 23)


In [5]:
'''
recordingnames_train = all_weeks_train['RecordingName'].unique()
recordingnames_test = all_weeks_test['RecordingName'].unique()
print(recordingnames_train)
print(recordingnames_test)
print('')

playernames_train = all_weeks_train['PlayerName'].unique()
playernames_test = all_weeks_test['PlayerName'].unique()
print(playernames_train)
print(playernames_test)
print('')

# drop vhil2 for later
# consolidate names for later

for playername in playernames_train:
    player_df = all_weeks_train[all_weeks_train['PlayerName'] == playername]
    player_test_df = all_weeks_test[all_weeks_test['PlayerName'] == playername]
    
    print(playername, player_df.shape[0], player_test_df.shape[0])
'''

['2021-06-30-1500' '2021-06-30-1600' '2021-06-30-1900' '2021-07-01-0900'
 '2021-07-01-1000' '2021-07-01-1500' '2021-07-01-1600' 'week1_1' 'week1_2'
 'week1_3' 'week1_4' 'week1_5' 'week1_6' 'week1_7' 'week1_8'
 '2021-07-07-1900' '2021-07-08-0900' '2021-07-08-1900' '2021-07-08-2000'
 'week2_1' 'week2_2' 'week2_3' 'week2_4_2' 'week2_4' 'week2_5' 'week2_6'
 'week2_7_2' 'week2_7' 'week2_8' '2021-07-14-1500' '2021-07-14-1600'
 '2021-07-14-1900' '2021-07-15-0900' '2021-07-15-1100' '2021-07-15-1500'
 '2021-07-15-1600' 'week3_1' 'week3_2' 'week3_3' 'week3_4' 'week3_6'
 'week3_7' 'week3_8' '2021-07-21-1500' '2021-07-21-1600' '2021-07-21-1900'
 '2021-07-22-0900' '2021-07-22-1000' '2021-07-22-1100' '2021-07-22-1500'
 '2021-07-22-1600' 'week4_1' 'week4_2' 'week4_3' 'week4_4' 'week4_5_2'
 'week4_5' 'week4_6' 'week4_7' 'week4_8' '2021-07-28-1500'
 '2021-07-28-1600' '2021-07-28-1900' '2021-07-29-0900' '2021-07-29-1000'
 '2021-07-29-1100' '2021-07-29-1500' '2021-07-29-1600' 'week5_2' 'week5_3'
 'week5_

In [3]:
translations = ['HeadPos_y', 'LeftHandPos_y', 'RightHandPos_y']

head_q0, head_q1, head_q2, head_q3 = all_weeks_train['HeadRot_w'], all_weeks_train['HeadRot_x'], all_weeks_train['HeadRot_y'], all_weeks_train['HeadRot_z'], 
lhand_q0, lhand_q1, lhand_q2, lhand_q3 = all_weeks_train['LeftHandRot_w'], all_weeks_train['LeftHandRot_x'], all_weeks_train['LeftHandRot_y'], all_weeks_train['LeftHandRot_z'], 
rhand_q0, rhand_q1, rhand_q2, rhand_q3 = all_weeks_train['RightHandRot_w'], all_weeks_train['RightHandRot_x'], all_weeks_train['RightHandRot_y'], all_weeks_train['RightHandRot_z'], 

all_weeks_train['head_alpha'] = np.arctan2(2*(head_q0*head_q3-head_q1*head_q2), 1-2*(head_q3**2+head_q1**2))
all_weeks_train['head_beta'] = np.arcsin(2*(-head_q0*head_q1-head_q2*head_q3))
all_weeks_train['head_gamma'] = np.arctan2(2*(head_q0*head_q2-head_q3*head_q1), 1-2*(head_q1**2+head_q2**2))

all_weeks_train['lhand_alpha'] = np.arctan2(2*(lhand_q0*lhand_q3-lhand_q1*lhand_q2), 1-2*(lhand_q3**2+lhand_q1**2))
all_weeks_train['lhand_beta'] = np.arcsin(2*(-lhand_q0*lhand_q1-lhand_q2*lhand_q3))
all_weeks_train['lhand_gamma'] = np.arctan2(2*(lhand_q0*lhand_q2-lhand_q3*lhand_q1), 1-2*(lhand_q1**2+lhand_q2**2))

all_weeks_train['rhand_alpha'] = np.arctan2(2*(rhand_q0*rhand_q3-rhand_q1*rhand_q2), 1-2*(rhand_q3**2+rhand_q1**2))
all_weeks_train['rhand_beta'] = np.arcsin(2*(-rhand_q0*rhand_q1-rhand_q2*rhand_q3))
all_weeks_train['rhand_gamma'] = np.arctan2(2*(rhand_q0*rhand_q2-rhand_q3*rhand_q1), 1-2*(rhand_q1**2+rhand_q2**2))

euler_angles = ['head_alpha', 'head_beta', 'head_gamma', 
                'lhand_alpha', 'lhand_beta', 'lhand_gamma', 
                'rhand_alpha', 'rhand_beta', 'rhand_gamma']

In [4]:
head_q0, head_q1, head_q2, head_q3 = all_weeks_test['HeadRot_w'], all_weeks_test['HeadRot_x'], all_weeks_test['HeadRot_y'], all_weeks_test['HeadRot_z'], 
lhand_q0, lhand_q1, lhand_q2, lhand_q3 = all_weeks_test['LeftHandRot_w'], all_weeks_test['LeftHandRot_x'], all_weeks_test['LeftHandRot_y'], all_weeks_test['LeftHandRot_z'], 
rhand_q0, rhand_q1, rhand_q2, rhand_q3 = all_weeks_test['RightHandRot_w'], all_weeks_test['RightHandRot_x'], all_weeks_test['RightHandRot_y'], all_weeks_test['RightHandRot_z'], 

all_weeks_test['head_alpha'] = np.arctan2(2*(head_q0*head_q3-head_q1*head_q2), 1-2*(head_q3**2+head_q1**2))
all_weeks_test['head_beta'] = np.arcsin(2*(-head_q0*head_q1-head_q2*head_q3))
all_weeks_test['head_gamma'] = np.arctan2(2*(head_q0*head_q2-head_q3*head_q1), 1-2*(head_q1**2+head_q2**2))

all_weeks_test['lhand_alpha'] = np.arctan2(2*(lhand_q0*lhand_q3-lhand_q1*lhand_q2), 1-2*(lhand_q3**2+lhand_q1**2))
all_weeks_test['lhand_beta'] = np.arcsin(2*(-lhand_q0*lhand_q1-lhand_q2*lhand_q3))
all_weeks_test['lhand_gamma'] = np.arctan2(2*(lhand_q0*lhand_q2-lhand_q3*lhand_q1), 1-2*(lhand_q1**2+lhand_q2**2))

all_weeks_test['rhand_alpha'] = np.arctan2(2*(rhand_q0*rhand_q3-rhand_q1*rhand_q2), 1-2*(rhand_q3**2+rhand_q1**2))
all_weeks_test['rhand_beta'] = np.arcsin(2*(-rhand_q0*rhand_q1-rhand_q2*rhand_q3))
all_weeks_test['rhand_gamma'] = np.arctan2(2*(rhand_q0*rhand_q2-rhand_q3*rhand_q1), 1-2*(rhand_q1**2+rhand_q2**2))

In [6]:
X_train = []
y_train = []

X_test = []
y_test = []

playernames_train = all_weeks_train['PlayerName'].unique()
playernames_test = all_weeks_test['PlayerName'].unique()

# dropped '2021-07-01-1000', causing too many issues
first_half = ['2021-06-30-1500', '2021-06-30-1600', '2021-06-30-1900', '2021-07-01-0900',
 '2021-07-01-1500', '2021-07-01-1600', 'week1_1', 'week1_2',
 'week1_3', 'week1_4', 'week1_5', 'week1_6', 'week1_7', 'week1_8',
 '2021-07-07-1900', '2021-07-08-0900', '2021-07-08-1900', '2021-07-08-2000',
 'week2_1', 'week2_2', 'week2_3', 'week2_4_2', 'week2_4', 'week2_5', 'week2_6',
 'week2_7_2', 'week2_7', 'week2_8', '2021-07-14-1500', '2021-07-14-1600',
 '2021-07-14-1900', '2021-07-15-0900', '2021-07-15-1100', '2021-07-15-1500',
 '2021-07-15-1600', 'week3_1', 'week3_2', 'week3_3', 'week3_4', 'week3_6',
 'week3_7', 'week3_8', '2021-07-21-1500', '2021-07-21-1600', '2021-07-21-1900',
 '2021-07-22-0900', '2021-07-22-1000', '2021-07-22-1100', '2021-07-22-1500',
 '2021-07-22-1600']

second_half = ['week4_1', 'week4_2', 'week4_3', 'week4_4', 'week4_5_2',
 'week4_5', 'week4_6', 'week4_7', 'week4_8', '2021-07-28-1500',
 '2021-07-28-1600', '2021-07-28-1900', '2021-07-29-0900', '2021-07-29-1000',
 '2021-07-29-1100', '2021-07-29-1500', '2021-07-29-1600', 'week5_2', 'week5_3',
 'week5_4', 'week5_5', 'week5_6', 'week5_7', 'week5_8', 'week5_A',
 '2021-08-11-1500', '2021-08-11-1600', '2021-08-11-1900', '2021-08-12-0900',
 '2021-08-12-1000', '2021-08-12-1100', '2021-08-12-1500', '2021-08-12-1600',
 'week6_1', 'week6_2', 'week6_3', 'week6_4', 'week6_5', 'week6_6', 'week6_7',
 'week6_8']

translations_euler_angles = translations + euler_angles

In [11]:
second_half = ['week4_1', 'week4_2', 'week4_3', 'week4_4', 'week4_5_2',
 'week4_5', 'week4_6', 'week4_7', 'week4_8', '2021-07-28-1500',
 '2021-07-28-1600', '2021-07-28-1900', '2021-07-29-0900', '2021-07-29-1000',
 '2021-07-29-1100', '2021-07-29-1500', '2021-07-29-1600', 'week5_2', 'week5_3',
 'week5_4', 'week5_5', 'week5_6', 'week5_7', 'week5_8', 'week5_A',
 '2021-08-11-1500', '2021-08-11-1600', '2021-08-11-1900', '2021-08-12-0900',
 '2021-08-12-1000', '2021-08-12-1100', '2021-08-12-1500', '2021-08-12-1600',
 'week6_1', 'week6_2', 'week6_3', 'week6_4', 'week6_5', 'week6_6', 'week6_7',
 'week6_8']

print('start time: ', time.strftime("%H:%M:%S", time.localtime()))

for recording_name in second_half:
    recording_df = all_weeks_train[all_weeks_train['RecordingName'] == recording_name]
    
    print(recording_name, 'start_time', time.strftime("%H:%M:%S", time.localtime()))
    
    players_recording = recording_df['PlayerName'].unique()
    
    for name in players_recording:
        player_df = recording_df[recording_df['PlayerName'] == name]
                    
        for i in range(15, player_df.shape[0], 30):
            features = []
            
            window_1s = None
            window_3s = None
            window_10s = None
            window_30s = None

            if i >= 15 and i < player_df.shape[0] - 14:
                window_1s = player_df.iloc[i - 15 : i + 14]
            if i >= 45 and i < player_df.shape[0] - 44:
                window_3s = player_df.iloc[i - 45 : i + 44]
            if i >= 150 and i < player_df.shape[0] - 149:
                window_10s = player_df.iloc[i - 150 : i + 149]
            if i >= 450 and i < player_df.shape[0] - 449:
                window_30s = player_df.iloc[i - 450 : i + 449]
                
            if window_1s is None or window_3s is None or window_10s is None or window_30s is None:
                continue
                

            for column_name in translations_euler_angles:
                for window in [window_1s, window_3s, window_10s, window_30s]:
                    column = window[column_name]
                    features.extend([column.max(), column.min(), column.median(), column.mean(), column.std()])
            
            for window in [window_1s, window_3s, window_10s, window_30s]:
                
                
                head_x_offset = window['HeadPos_x'] - window['HeadPos_x'].mean()
                head_z_offset = window['HeadPos_z'] - window['HeadPos_z'].mean()
                head_angle = -circmean(window['head_gamma'], high=np.pi, low=-np.pi)
                head_surging = head_z_offset * np.cos(head_angle) - head_x_offset * np.sin(head_angle)
                head_strafing = head_z_offset * np.sin(head_angle) + head_x_offset * np.cos(head_angle)
                
                features.extend([head_surging.max(), head_surging.min(), head_surging.median(), head_surging.std()])
                features.extend([head_strafing.max(), head_strafing.min(), head_strafing.median(), head_strafing.std()])
                
                lhand_x_offset = window['LeftHandPos_x'] - window['LeftHandPos_x'].mean()
                lhand_z_offset = window['LeftHandPos_z'] - window['LeftHandPos_z'].mean()
                lhand_angle = -circmean(window['lhand_gamma'], high=np.pi, low=-np.pi)
                lhand_surging = lhand_z_offset * np.cos(lhand_angle) - lhand_x_offset * np.sin(lhand_angle)
                lhand_strafing = lhand_z_offset * np.sin(lhand_angle) + lhand_x_offset * np.cos(lhand_angle)
                
                features.extend([lhand_surging.max(), lhand_surging.min(), lhand_surging.median(), lhand_surging.std()])
                features.extend([lhand_strafing.max(), lhand_strafing.min(), lhand_strafing.median(), lhand_strafing.std()])
                
                rhand_x_offset = window['RightHandPos_x'] - window['RightHandPos_x'].mean()
                rhand_z_offset = window['RightHandPos_z'] - window['RightHandPos_z'].mean()
                rhand_angle = -circmean(window['rhand_gamma'], high=np.pi, low=-np.pi)
                rhand_surging = rhand_z_offset * np.cos(rhand_angle) - rhand_x_offset * np.sin(rhand_angle)
                rhand_strafing = rhand_z_offset * np.sin(rhand_angle) + rhand_x_offset * np.cos(rhand_angle)
                
                features.extend([rhand_surging.max(), rhand_surging.min(), rhand_surging.median(), rhand_surging.std()])
                features.extend([rhand_strafing.max(), rhand_strafing.min(), rhand_strafing.median(), rhand_strafing.std()])
                
                
            X_train.append(features)
            y_train.append(np.where(playernames_train == name)[0][0])
                
print('end time: ', time.strftime("%H:%M:%S", time.localtime()))

print(len(X_train))
print(len(X_train[0]))
print(len(y_train))


with open('random_forest/X_train_all.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(X_train)

with open('random_forest/y_train_all.txt', 'w') as f:
    f.write('\n'.join(str(y) for y in y_train))

start time:  01:52:51
week4_1 start_time 01:52:55
week4_2 start_time 01:59:54
week4_3 start_time 02:05:17
week4_4 start_time 02:11:50
week4_5_2 start_time 02:30:02
week4_5 start_time 02:30:04
week4_6 start_time 02:31:13
week4_7 start_time 02:39:07
week4_8 start_time 02:44:45
2021-07-28-1500 start_time 03:02:44
2021-07-28-1600 start_time 03:14:13
2021-07-28-1900 start_time 03:40:29
2021-07-29-0900 start_time 03:48:41
2021-07-29-1000 start_time 04:01:07
2021-07-29-1100 start_time 04:25:52
2021-07-29-1500 start_time 04:44:55
2021-07-29-1600 start_time 05:08:23
week5_2 start_time 05:29:45
week5_3 start_time 05:44:08
week5_4 start_time 06:03:05
week5_5 start_time 06:17:46
week5_6 start_time 06:37:00
week5_7 start_time 06:58:00
week5_8 start_time 07:21:08
week5_A start_time 07:46:15
2021-08-11-1500 start_time 08:04:44
2021-08-11-1600 start_time 08:17:58
2021-08-11-1900 start_time 08:32:55
2021-08-12-0900 start_time 08:52:56
2021-08-12-1000 start_time 09:04:32
2021-08-12-1100 start_time 09:24

727417
336
727417

1350233
336
1350233

In [12]:
print('start time: ', time.strftime("%H:%M:%S", time.localtime()))

recordingnames_test = all_weeks_test['RecordingName'].unique()

for recording_name in recordingnames_test:
    recording_df = all_weeks_test[all_weeks_test['RecordingName'] == recording_name]
    
    print(recording_name, 'start_time', time.strftime("%H:%M:%S", time.localtime()))
    
    players_recording = recording_df['PlayerName'].unique()
    
    for name in players_recording:
        player_df = recording_df[recording_df['PlayerName'] == name]
        
        print(name, 'start time', time.strftime("%H:%M:%S", time.localtime()))
                    
        for i in range(15, player_df.shape[0], 30):
            features = []
            
            window_1s = None
            window_3s = None
            window_10s = None
            window_30s = None

            if i >= 15 and i < player_df.shape[0] - 14:
                window_1s = player_df.iloc[i - 15 : i + 14]
            if i >= 45 and i < player_df.shape[0] - 44:
                window_3s = player_df.iloc[i - 45 : i + 44]
            if i >= 150 and i < player_df.shape[0] - 149:
                window_10s = player_df.iloc[i - 150 : i + 149]
            if i >= 450 and i < player_df.shape[0] - 449:
                window_30s = player_df.iloc[i - 450 : i + 449]
                
            if window_1s is None or window_3s is None or window_10s is None or window_30s is None:
                continue
                

            for column_name in translations_euler_angles:
                for window in [window_1s, window_3s, window_10s, window_30s]:
                    column = window[column_name]
                    features.extend([column.max(), column.min(), column.median(), column.mean(), column.std()])
            
            for window in [window_1s, window_3s, window_10s, window_30s]:
                head_x_offset = window['HeadPos_x'] - window['HeadPos_x'].mean()
                head_z_offset = window['HeadPos_z'] - window['HeadPos_z'].mean()
                head_angle = -circmean(window['head_gamma'], high=np.pi, low=-np.pi)
                head_surging = head_z_offset * np.cos(head_angle) - head_x_offset * np.sin(head_angle)
                head_strafing = head_z_offset * np.sin(head_angle) + head_x_offset * np.cos(head_angle)
                
                features.extend([head_surging.max(), head_surging.min(), head_surging.median(), head_surging.std()])
                features.extend([head_strafing.max(), head_strafing.min(), head_strafing.median(), head_strafing.std()])
                
                lhand_x_offset = window['LeftHandPos_x'] - window['LeftHandPos_x'].mean()
                lhand_z_offset = window['LeftHandPos_z'] - window['LeftHandPos_z'].mean()
                lhand_angle = -circmean(window['lhand_gamma'], high=np.pi, low=-np.pi)
                lhand_surging = lhand_z_offset * np.cos(lhand_angle) - lhand_x_offset * np.sin(lhand_angle)
                lhand_strafing = lhand_z_offset * np.sin(lhand_angle) + lhand_x_offset * np.cos(lhand_angle)
                
                features.extend([lhand_surging.max(), lhand_surging.min(), lhand_surging.median(), lhand_surging.std()])
                features.extend([lhand_strafing.max(), lhand_strafing.min(), lhand_strafing.median(), lhand_strafing.std()])
                
                rhand_x_offset = window['RightHandPos_x'] - window['RightHandPos_x'].mean()
                rhand_z_offset = window['RightHandPos_z'] - window['RightHandPos_z'].mean()
                rhand_angle = -circmean(window['rhand_gamma'], high=np.pi, low=-np.pi)
                rhand_surging = rhand_z_offset * np.cos(rhand_angle) - rhand_x_offset * np.sin(rhand_angle)
                rhand_strafing = rhand_z_offset * np.sin(rhand_angle) + rhand_x_offset * np.cos(rhand_angle)
                
                features.extend([rhand_surging.max(), rhand_surging.min(), rhand_surging.median(), rhand_surging.std()])
                features.extend([rhand_strafing.max(), rhand_strafing.min(), rhand_strafing.median(), rhand_strafing.std()])
                
                
            X_test.append(features)
            y_test.append(np.where(playernames_test == name)[0][0])
                
print('end time: ', time.strftime("%H:%M:%S", time.localtime()))

print(len(X_test))
print(len(X_test[0]))
print(len(y_test))

with open('random_forest/X_test_all.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(X_test)
    
with open('random_forest/y_test_all.txt', 'w') as f:
    f.write('\n'.join(str(y) for y in y_test))

start time:  19:12:57
2021-08-18-1500 start_time 19:13:32
ADE start time 19:13:32
Parker start time 19:14:00
Caitlin start time 19:16:59
Brendan start time 19:17:20
Pujan start time 19:17:49
Mauricio start time 19:18:08
Vivek start time 19:18:14
Angela start time 19:18:44
kristina start time 19:19:15
2021-08-18-1600 start_time 19:22:03
Leopold start time 19:22:03
Kasha  2 start time 19:23:05
Brian start time 19:24:04
Tayden start time 19:27:32
Michelle start time 19:28:27
Shana start time 19:32:04
Ofure start time 19:33:12
2021-08-18-1900 start_time 19:36:25
Isabelle start time 19:36:25
Julia start time 19:37:07
Alex start time 19:37:44
Emily start time 19:38:25
Jordan start time 19:41:32
Philip start time 19:42:18
Allison start time 19:42:56
John start time 19:43:41
Dax start time 19:46:43
Rishab start time 19:47:18
Jake start time 19:47:19
justin start time 19:48:06
Alonzo start time 19:51:20
2021-08-19-0900 start_time 19:52:08
Ayoade start time 19:52:08
Alexis start time 19:52:57
An

Thomas start time 03:14:00
Chloe start time 03:18:17
Zahran start time 03:19:12
Annie start time 03:20:05
Nick start time 03:24:10
Donjae start time 03:25:08
Jason start time 03:26:05
Sunmi start time 03:27:01
week8_6 start_time 03:31:20
Addie start time 03:31:20
Takao start time 03:31:20
Nicolas start time 03:31:20
EJ Smith start time 03:31:20
Nick start time 03:31:20
Donjae start time 03:31:20
Jason start time 03:31:20
Sunmi start time 03:31:20
Zahran start time 03:31:20
week8_7 start_time 03:31:24
Bryan start time 03:31:24
juliette start time 03:32:35
Miranda start time 03:33:27
Pedro start time 03:37:58
adrian start time 03:39:10
Richard start time 03:43:44
Ember start time 03:44:24
Finn start time 03:45:35
Jess start time 03:45:35
Rafael start time 03:46:44
Ugyen  Lama start time 03:51:14
Thomas start time 03:52:25
week8_8_2 start_time 03:56:59
Lynn start time 03:56:59
Chu start time 03:57:00
Sharon start time 03:57:01
Roshini start time 03:57:02
Dante start time 03:57:03
Kimmy st

In [19]:
import re

lines = []
with open('random_forest/y_test_all.txt') as f:
    lines = f.readlines()
    
for i in range(len(lines)):
    lines[i] = re.sub(r'\D+', '', lines[i])
    
    
y_test = lines
with open('random_forest/y_test_all_2.txt', 'w') as f:
    f.write('\n'.join(str(y) for y in y_test))
    
    
print(len(y_test))

456822
456822


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_train_fitted = scaler.fit_transform(X_train)
X_test_fitted = scaler.transform(X_test)

# gini, min_samples_leaf: 30, n_estimators: 100, max_depth: None
print('start time: ', time.strftime("%H:%M:%S", time.localtime()))
classifier = RandomForestClassifier(n_estimators=100, criterion='gini', min_samples_leaf=30, max_depth=None, random_state = 42)
classifier.fit(X_train_fitted, y_train)
print('end time: ', time.strftime("%H:%M:%S", time.localtime()))

start time:  14:48:03
end time:  16:33:36
0.0


/opt/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:217: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


In [21]:
for i in range(len(y_test)):
    y_test[i] = int(y_test[i])

y_pred = classifier.predict(X_test_fitted)
print(accuracy_score(y_test, y_pred))

0.01690811738488952


In [22]:
playernames_train = all_weeks_train['PlayerName'].unique()
playernames_test = all_weeks_test['PlayerName'].unique()
print(len(playernames_train))
print(len(playernames_test))

111
85


In [23]:
recordingnames_train = all_weeks_train['RecordingName'].unique()

players_per_session = []

for recording_name in second_half:
    recording_df = all_weeks_train[all_weeks_train['RecordingName'] == recording_name]
    
    players_recording = recording_df['PlayerName'].unique()
    players_per_session.append(len(players_recording))

print(players_per_session)

[7, 10, 12, 8, 12, 12, 12, 14, 9, 8, 12, 9, 7, 13, 12, 12, 11, 10, 9, 7, 11, 12, 12, 11, 9, 8, 9, 13, 5, 10, 14, 14, 9, 8, 9, 12, 5, 10, 15, 14, 9]


In [24]:
print(np.where(y_test == 0)[0])

[]


In [26]:
# run with 1 through 10 trees
print('start time: ', time.strftime("%H:%M:%S", time.localtime()))
classifier_1 = RandomForestClassifier(n_estimators=1, criterion='gini', min_samples_leaf=30, max_depth=None, random_state = 42)
classifier_1.fit(X_train_fitted, y_train)
print(accuracy_score(y_test, classifier_1.predict(X_test_fitted)))
print('end time: ', time.strftime("%H:%M:%S", time.localtime()))

start time:  10:54:29
0.014123663046000412
end time:  10:56:21


predictions at level per people/session and feature importance